# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:
- Practice your SQL knowledge


## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [3]:
import pandas as pd
import sqlite3
conn= sqlite3.connect('data.sqlite')
c= conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, and address for all customers in California with a credit limit greater than 25000.00. 

In [14]:
# For the first query, the boilerplate for getting the query into a dataframe has been provided for you
c.execute("""select contactFirstName, contactLastName, phone, addressLine1, creditLimit
            from customers
            where state = 'CA' and creditLimit > '25000.00'
            ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Julie,Murphy,6505555787,5557 North Pendale Street,64600.00
1,Juri,Hashimoto,6505556809,9408 Furth Circle,84600.00
2,Julie,Young,6265557265,78934 Hillside Dr.,90700.00
3,Brian,Chandler,2155554369,6047 Douglas Av.,57700.00
4,Sue,Frick,4085553659,3086 Ingle Ln.,77600.00
5,Steve,Thompson,3105553722,3675 Furth Circle,55400.00
6,Sue,Taylor,4155554312,2793 Furth Circle,60300.00


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [19]:
c.execute("""select state, country, avg(creditLimit) as avg_cr
            from customers
            where country = 'USA'
            group by state;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,country,avg_cr
0,CA,USA,83854.545455
1,CT,USA,57350.000000
2,MA,USA,70755.555556
3,NH,USA,114200.000000
4,NJ,USA,43000.000000
5,NV,USA,71800.000000
6,NY,USA,89966.666667
7,PA,USA,84766.666667


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [28]:
c.execute(""" select customerName, customerNumber, orderNumber, status, sum(quantityOrdered)
            from customers
            join orders
            using(customerNumber)
            join orderDetails
            using (orderNumber)
            group by 1""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,orderNumber,status,sum(quantityOrdered)
0,"AV Stores, Co.",187,10332,Shipped,1778
1,Alpha Cognac,242,10397,Shipped,687
2,Amica Models & Co.,249,10293,Shipped,843
3,"Anna's Decorations, Ltd",276,10391,Shipped,1469
4,Atelier graphique,103,10345,Shipped,270


#### Expected Output

<img src='images/expected-output-3.png'>

## HAVING and ORDER BY

Now, repeat the last query, but only get orders from customers that have a quantityOrdered value greater than 30. Sort the rows in ascending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [34]:
c.execute(""" select customerName, customerNumber, orderNumber, status, count(quantityOrdered)
            from customers
            join orders
            using(customerNumber)
            join orderDetails
            using (orderNumber)
            group by 1
            having count(quantityOrdered) > 40
            order by count(quantityOrdered)""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerName,customerNumber,orderNumber,status,count(quantityOrdered)
0,Corporate Gift Ideas Co.,321,10384,Shipped,41
1,Reims Collectables,353,10398,Shipped,41
2,"Saveley & Henriot, Co.",146,10227,Shipped,41
3,"Dragon Souveniers, Ltd.",148,10387,Shipped,43
4,"Anna's Decorations, Ltd",276,10391,Shipped,46
5,"Down Under Souveniers, Inc",323,10404,Shipped,46
6,Souveniers And Things Co.,282,10420,In Process,46
7,Kelly's Gift Shop,496,10399,Shipped,48
8,Muscle Machine Inc,151,10349,Shipped,48
9,Rovelli Gifts,278,10328,Shipped,48


#### Expected Output

<img src='images/expected-output-4.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [39]:
c.execute("""select lastName, firstName, employeeNumber, officeCode
                    FROM employees
                    WHERE officeCode IN (SELECT officeCode 
                                                FROM offices 
                                                JOIN employees
                                                USING(officeCode)
                                                GROUP BY 1
                                                HAVING COUNT(employeeNumber) < 5
                                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3
5,Bott,Larry,1501,7
6,Jones,Barry,1504,7
7,Fixter,Andy,1611,6
8,Marsh,Peter,1612,6
9,King,Tom,1619,6


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!